In [173]:
from bs4 import BeautifulSoup
import folium
from folium import plugins

import pandas as pd
import requests
import numpy as np


In [154]:
def get_lat_and_lon(location, key="512d512d3f82ab80a8697bc138f2fa2a"):

    json_body = {
        "address": location,
        "output": "XML", 
        "key": key,
        "batch": True if "|" in location else False # every 10 locations will be batching togehter
    }
    r = requests.post(
        "https://restapi.amap.com/v3/geocode/geo", data = json_body)

    lats_and_lons = []
    for geocode in r.json()["geocodes"]:
       lats_and_lons.append(list(map(float, geocode["location"].split(",")))) 
    
    return lats_and_lons

In [11]:
def PlotLineOnMap(lats_and_lons):
    # 给出的坐标系为GCJ-02，如果需要测试google地图，需要进行坐标转换
    tri = np.array(lats_and_lons)
    print(tri)
    # return

    san_map = folium.Map(
        location=[121.500766, 31.150915],
        zoom_start=16,
        # 高德街道图
        tiles='http://webrd02.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=7&x={x}&y={y}&z={z}',
        # tiles='http://webst02.is.autonavi.com/appmaptile?style=6&x={x}&y={y}&z={z}', # 高德卫星图
        attr='default')

    # from coord_convert.transform import wgs2gcj, wgs2bd, gcj2wgs, gcj2bd, bd2wgs, bd2gcj 
    # lon, lat = 121.500766, 31.150915
    # gcj_lon, gcj_lat = gcj2wgs(lon, lat)


    # folium.PolyLine(tri, color='#3388ff').add_to(san_map)
    # marker_cluster = plugins.MarkerCluster().add_to(san_map)
    # for lat, lon in lats_and_lons:
    #     folium.Marker([lat, lon], color='red').add_to(marker_cluster)
    san_map.save('/Users/pengyuyan/Downloads/test.html')


def main():
    PlotLineOnMap(lats_and_lons)


if __name__ == '__main__':
    main()



[[121.551406  31.168369]
 [121.599269  31.273567]
 [121.497626  31.133457]
 [121.550328  31.165054]]


In [129]:

def get_city_disease_info(
    shanghaifabu_url="https://mp.weixin.qq.com/s/w8UqtdmBtdLQitM7emOVjw", city_name="上海市"):

    r = requests.get(shanghaifabu_url)
    demo = r.text

    soup = BeautifulSoup(demo, 'html.parser')
    location_div = soup.find("div", attrs={"class": "rich_media_content"})
    locate_section_div = location_div.findAll("section", attrs={"data-role": "title"})

    city_qu_jiedao = {f"{city_name}": []}
    for i, sub_div in enumerate(locate_section_div):
        area_name = sub_div.find("strong").text
        ps = sub_div.findAll("p")

        area_xiaoqu = []
        for j, area_ps in enumerate(ps):        
            if j <= 1 or j >= len(ps)-2:
                continue

            area_xiaoqu.append(f"{city_name}"+area_name+area_ps.text.strip("，"))
        city_qu_jiedao[f"{city_name}"].append({area_name: area_xiaoqu})


In [132]:
def spilt_jiedaos_into_batch(jiedaos, n = 10):
    return [jiedaos[i:i + n] for i in range(0, len(jiedaos), n)] 

In [172]:
def save_info_to_dir():
    city_qu_jiedao = get_city_disease_info()
    shanghai_info = city_qu_jiedao["上海市"]

    dfs = []
    for q, qu in enumerate(shanghai_info):

        df_list = []
        for key, value in qu.items():

            jiedaos = value
            batch_jiedaos = spilt_jiedaos_into_batch(jiedaos)
            lats_and_lons = []
            for batch in batch_jiedaos:
                address = "|".join(batch)
                lats_and_lons.extend(get_lat_and_lon(address))
        
        for i, (jiedao, lat_and_lon) in enumerate(list(zip(jiedaos, lats_and_lons))):
            df_list.append({
                "ch_name": jiedao,
                "city": "上海",
                "lat_and_lon": ",".join(map(str, list(lats_and_lons[i]))) 
            })     
        
        df = pd.DataFrame(df_list, columns=df_list[0].keys())
        df.to_csv(f"~/{key}.csv", index=False, encoding='gbk')

        dfs.extend(df_list) 
    
    df = pd.DataFrame(dfs, columns=dfs[0].keys())
    df.to_csv(f"~/shanghai.csv", index=False, encoding='gbk')